In [1]:
import pandas as pd
import torch

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_values = train_df.values
test_values = test_df.values

trainX = torch.tensor(train_values[:, :-1], dtype=torch.float64)
trainY = torch.tensor(train_values[:, -1:], dtype=torch.float64)
testX = torch.tensor(test_values[:, :-1], dtype=torch.float64)
testY = torch.tensor(test_values[:, -1:], dtype=torch.float64)

In [3]:
#q1
bytes = trainX.element_size()
elements = trainX.nelement() 
total = bytes * elements
total

83520

In [4]:
#q2
trainX_float16 = trainX.to(torch.float16)
trainX_float64 = trainX_float16.to(torch.float64)

difference = torch.abs(trainX - trainX_float64)
max_difference = torch.max(difference).item()
max_difference

0.0

In [5]:
#q3
cuda = torch.cuda.is_available()
cuda

False

In [6]:
coef = torch.tensor([
    [0.0040],
    [0.0040],
    [0.0040],
    [0.0040],
    [0.0040],
    [0.0040],
    [0.0300],
    [0.0300],
    [0.0300],
    [0.0300]
], dtype=torch.float64)

In [7]:
#q4
predicted_deaths = (testX[0] * coef.squeeze()).sum().item()
predicted_deaths

9.844

In [8]:
#q5
predicted_deaths_avg = (testX @ coef).mean().item()
predicted_deaths_avg

12.073632183908048

In [9]:
#q6
x = torch.tensor(0.0, requires_grad=True)
y = x**2 - 8*x + 19
float(y)

19.0

In [10]:
#q7
x = torch.tensor([0.0], requires_grad=True)
optimizer = torch.optim.SGD([x], lr=0.01)

num_iterations = 1000
for _ in range(num_iterations):
    y = x**2 - 8*x + 19
    optimizer.zero_grad()
    y.backward()
    optimizer.step()

optimized_x = x.detach().item()
optimized_x

3.9999940395355225

In [11]:
#q8
num_features = trainX.size(1)
coef = torch.zeros(num_features, 1, dtype=trainX.dtype)
predictions = trainX @ coef
mse = torch.nn.functional.mse_loss(predictions, trainY)
mse.item()

197.8007662835249

In [12]:
ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)

In [13]:
#q9
torch.manual_seed(544)
coef = torch.zeros(num_features, 1, dtype=trainX.dtype, requires_grad=True)
optimizer = torch.optim.SGD([coef], lr=0.000002)
loss_fn = torch.nn.MSELoss()

epochs = 500
for epoch in range(epochs):
    for batch_x, batch_y in dl:
        predictions = batch_x @ coef
        loss = loss_fn(predictions, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

predictions = trainX @ coef
mse_train = loss_fn(predictions, trainY).item()
mse_train

26.8113940147193

In [14]:
#q10
predictions = testX @ coef
mse_test = loss_fn(predictions, testY).item()
mse_test

29.05854692548551